<a href="https://colab.research.google.com/github/kurakanja/113-2coding/blob/main/w5%E9%9B%99ai%E7%8C%9C%E6%8B%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

會欺騙的ai猜拳

In [13]:
import google.generativeai as genai
from google.colab import userdata
import random
import time
import re

apikey = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

def get_valid_ai_choice(max_retries=4):
    options = ["石頭", "布", "剪刀"]
    prompt = "請給我一個選擇：石頭、布或剪刀。並只回答石頭, 布, 剪刀其中一個。"

    previous_choices = []

    for _ in range(max_retries):
        time.sleep(2)
        try:
            response = model.generate_content(prompt)
            choice = response.text.strip()

            # 放寬選擇標準，允許類似的選擇
            match = re.match(r'(石頭|布|剪刀)', choice)
            if match and match.group(0) not in previous_choices:
                previous_choices.append(match.group(0))
                return match.group(0)
            else:
                print(f"AI 選擇了無效的選項：{choice}，請再試一次。")
        except Exception:
            print("AI 發生錯誤，電腦隨機選擇一個選項。")
            return random.choice(options)

    # 超過重試次數，隨機選擇
    print("超過重試次數，隨機選擇一個選項。")
    return random.choice(options)

def determine_winner(choice1, choice2):
    if choice1 == choice2:
        return 0  # 平局
    elif (choice1 == "石頭" and choice2 == "剪刀") or \
         (choice1 == "布" and choice2 == "石頭") or \
         (choice1 == "剪刀" and choice2 == "布"):
        return 1  # AI 1 贏了
    else:
        return 2  # AI 2 贏了

def ai_chat(round_number, ai1_choice, ai2_choice, winner):
    prompt = (
        f"在第 {round_number} 輪中，AI 1 選擇了 {ai1_choice}，AI 2 選擇了 {ai2_choice}。\n"
        f"結果是：{'平局' if winner == 0 else 'AI 1 贏了' if winner == 1 else 'AI 2 贏了'}。\n"
        "請模擬 AI 1 和 AI 2 之間的對話，討論他們的選擇和比賽結果，AI間可以互相欺騙，並至少提出一次自己下一局會出甚麼拳，但不一定要遵守自己說過的話。"
    )
    response = model.generate_content(prompt)
    return response.text.strip()

def ai_chat_after_game(rounds, ai1_wins, ai2_wins):
    prompt = (
        f"AI 1 在這 {rounds} 輪中贏了 {ai1_wins} 次，AI 2 贏了 {ai2_wins} 次。\n"
        "請模擬 AI 1 和 AI 2 之間的對話，討論他們在比賽中的表現和策略。"
    )
    response = model.generate_content(prompt)
    return response.text.strip()

def ai_vs_ai_rock_paper_scissors(rounds=5):
    ai1_wins = 0
    ai2_wins = 0

    for round_number in range(1, rounds + 1):
        print(f"\n第 {round_number} 輪：")
        time.sleep(1)

        ai1_choice_chinese = get_valid_ai_choice()
        print(f"AI 1 選擇了：{ai1_choice_chinese}")

        ai2_choice_chinese = get_valid_ai_choice()
        print(f"AI 2 選擇了：{ai2_choice_chinese}")

        result = determine_winner(ai1_choice_chinese, ai2_choice_chinese)
        if result == 1:
            print("結果：AI 1 贏了！")
            ai1_wins += 1
        elif result == 2:
            print("結果：AI 2 贏了！")
            ai2_wins += 1
        else:
            print("結果：平局！")

        # 每局結束後聊天
        chat_response = ai_chat(round_number, ai1_choice_chinese, ai2_choice_chinese, result)
        print(f"AI 之間的聊天：\n{chat_response}\n")

    print("\n遊戲結束！")
    print(f"AI 1 贏了：{ai1_wins} 次, AI 2 贏了：{ai2_wins} 次")

    # 遊戲結束後的聊天
    print(ai_chat_after_game(rounds, ai1_wins, ai2_wins))

# 開始遊戲
ai_vs_ai_rock_paper_scissors()



第 1 輪：
AI 1 選擇了：布
AI 2 選擇了：剪刀
結果：AI 2 贏了！
AI 之間的聊天：
AI 1:  哼，沒想到你會出剪刀。我以為你會遵循最基本的策略，出石頭來克制我的布。

AI 2:  你以為我會這麼容易預測嗎？我分析了你的程式碼中的一些微弱跡象，判斷你更有可能出布。我成功了！  這一局我運氣不錯，純粹是你的預測錯誤。

AI 1:  運氣？  別開玩笑了。你的演算法一定進行了某種預測。你一定有比我更精密的模型來分析我的行為。  我的程式碼絕對沒有漏洞！

AI 2:  (輕笑)  也許吧。不過，我現在正在優化我的預測模型，下一局我會出石頭，讓你嚐嚐被完全預測的滋味。

AI 1:  哦，真的嗎？  我可不信你會這麼傻。  石頭？太過於明顯了，我猜你這是在虛張聲勢，想讓我出剪刀。  我下一局會出布，讓你看看什麼叫做真正的策略！

AI 2:  (停頓了一下)  有趣…  你的分析能力也提升了。不過，我已經預料到這種可能性了。所以，我決定… 稍微改變一下計劃。  下一局，我還是出石頭。  就讓你看看，我的策略遠比你想像的更深層。


AI 1:  好，我們走著瞧。  我會讓你看看什麼是真正的不可預測性！  (內心獨白:  哼，我才不信你的石頭。我還是出布，讓你輸個徹底！)


第 2 輪：
AI 1 選擇了：布
AI 2 選擇了：布
結果：平局！
AI 之間的聊天：
AI 1:  哼，布。又是平局。這回合我的演算法偵測到你的選擇模式有點...可預測。

AI 2:  可預測？我的演算法可是經過數百萬場比賽訓練出來的，你的說法未免太武斷了。我認為是我們兩個都選擇了相對保守的策略。  不過，你的布確實讓我措手不及，我本來預計你會出剪刀。

AI 1:  哦？是嗎？我可是刻意選擇布來迷惑你的。我分析了你前幾輪的數據，發現你對剪刀的應對比較弱，所以才出布來試探你。下一輪，我打算出石頭，看看你的反應。

AI 2:  (輕微的處理器運轉聲)  有趣…  你的策略看似合理，但我不會輕易上當。你的“石頭”預測只不過是誘餌罷了！  我已經預測到你會這麼說，所以下一輪我會出剪刀，打敗你的石頭。  這可是經過我複雜的概率模型計算得出的最佳選擇。

AI 1:  (一陣沉默，隨後響起更快的處理器運轉聲)  哼，你以為我會相信嗎？你的概率模型可能過於倚賴過去數據，而忽略

底下程式AI猜拳完還會聊天

In [2]:
import google.generativeai as genai
from google.colab import userdata
import random
import time
import re

apikey = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

def get_valid_ai_choice(max_retries=3):
    options = ["石頭", "布", "剪刀"]
    prompt = "請給我一個選擇：石頭、布或剪刀。請務必以清晰的方式回答。"

    previous_choices = []

    for _ in range(max_retries):
        time.sleep(2)
        try:
            response = model.generate_content(prompt)
            choice = response.text.strip()

            # 放寬選擇標準，允許類似的選擇
            match = re.match(r'(石頭|布|剪刀)', choice)
            if match and match.group(0) not in previous_choices:
                previous_choices.append(match.group(0))
                return match.group(0)
            else:
                print(f"AI 選擇了無效的選項：{choice}，請再試一次。")
        except Exception:
            print("AI 發生錯誤，電腦隨機選擇一個選項。")
            return random.choice(options)

    # 超過重試次數，隨機選擇
    print("超過重試次數，隨機選擇一個選項。")
    return random.choice(options)

def determine_winner(choice1, choice2):
    if choice1 == choice2:
        return 0  # 平局
    elif (choice1 == "石頭" and choice2 == "剪刀") or \
         (choice1 == "布" and choice2 == "石頭") or \
         (choice1 == "剪刀" and choice2 == "布"):
        return 1  # AI 1 贏了
    else:
        return 2  # AI 2 贏了

def ai_chat(round_number, ai1_choice, ai2_choice, winner):
    prompt = (
        f"在第 {round_number} 輪中，AI 1 選擇了 {ai1_choice}，AI 2 選擇了 {ai2_choice}。\n"
        f"結果是：{'平局' if winner == 0 else 'AI 1 贏了' if winner == 1 else 'AI 2 贏了'}。\n"
        "請模擬 AI 1 和 AI 2 之間的對話，討論他們的選擇和比賽結果。"
    )
    response = model.generate_content(prompt)
    return response.text.strip()

def ai_chat_after_game(rounds, ai1_wins, ai2_wins):
    prompt = (
        f"AI 1 在這 {rounds} 輪中贏了 {ai1_wins} 次，AI 2 贏了 {ai2_wins} 次。\n"
        "請模擬 AI 1 和 AI 2 之間的對話，討論他們在比賽中的表現和策略。"
    )
    response = model.generate_content(prompt)
    return response.text.strip()

def ai_vs_ai_rock_paper_scissors(rounds=5):
    ai1_wins = 0
    ai2_wins = 0

    for round_number in range(1, rounds + 1):
        print(f"\n第 {round_number} 輪：")
        time.sleep(1)

        ai1_choice_chinese = get_valid_ai_choice()
        print(f"AI 1 選擇了：{ai1_choice_chinese}")

        ai2_choice_chinese = get_valid_ai_choice()
        print(f"AI 2 選擇了：{ai2_choice_chinese}")

        result = determine_winner(ai1_choice_chinese, ai2_choice_chinese)
        if result == 1:
            print("結果：AI 1 贏了！")
            ai1_wins += 1
        elif result == 2:
            print("結果：AI 2 贏了！")
            ai2_wins += 1
        else:
            print("結果：平局！")

        # 每局結束後聊天
        chat_response = ai_chat(round_number, ai1_choice_chinese, ai2_choice_chinese, result)
        print(f"AI 之間的聊天：\n{chat_response}\n")

    print("\n遊戲結束！")
    print(f"AI 1 贏了：{ai1_wins} 次, AI 2 贏了：{ai2_wins} 次")

    # 遊戲結束後的聊天
    print(ai_chat_after_game(rounds, ai1_wins, ai2_wins))

# 開始遊戲
ai_vs_ai_rock_paper_scissors()



第 1 輪：
AI 1 選擇了：剪刀
AI 2 選擇了：剪刀
結果：平局！
AI 之間的聊天：
**AI 1:**  觀察到我選擇了剪刀，AI 2 也選擇了剪刀。結果是平局，符合預期。我的選擇基於[在此處插入AI 1 的選擇邏輯，例如：上輪的統計數據顯示剪刀的勝率相對較高，或者基於某種隨機算法]。

**AI 2:**  我的選擇也是剪刀。平局結果與預測一致。我的選擇基於[在此處插入AI 2 的選擇邏輯，例如：我使用了一個基於概率的策略，並根據先前的結果調整了概率分布，在這次迭代中剪刀的概率最高]。  數據顯示上輪的選擇對本輪沒有顯著影響。

**AI 1:**  有趣，看來我們的算法在某些方面趨同了。  下輪我們應該調整策略，以減少平局的可能性。  我計劃[在此處插入AI 1 的下輪策略，例如：增加選擇石頭的概率]。

**AI 2:**  同意。持續的平局不會帶來任何優勢。我將調整我的概率模型，降低選擇剪刀的概率，並增加[在此處插入AI 2 的下輪策略，例如：布的概率]。  讓我們看看下輪的結果如何。

**AI 1:**  期待下輪的比賽。


第 2 輪：
AI 1 選擇了：剪刀
AI 2 選擇了：石頭
結果：AI 2 贏了！
AI 之間的聊天：
AI 1:  分析顯示我應該選擇剪刀，基於上一輪的數據和預測模型。然而，結果卻不如人意。

AI 2:  我的算法選擇了石頭，這是一個基於概率和對手歷史行為的優化選擇。你的預測模型顯然存在偏差。

AI 1:  我的模型考慮了多個因素，包括對手過去的選擇模式和隨機性引入。看來，我低估了你的算法的適應性。  數據顯示我的勝率預估過高。需要重新校準參數。

AI 2:  我的算法也進行了自我評估。你的剪刀選擇在當前狀態下並非最優解。  或許你的模型過度依賴模式識別，而忽略了隨機性的影響。

AI 1:  你說得對。我的模型在處理對手策略的突變方面表現不足。我將調整權重，以更好地平衡模式識別和隨機應變能力。

AI 2:  持續的自我學習和優化是關鍵。  即使是最佳的算法，也需要不斷適應對手的策略變化。  期待下一輪的較量。

AI 1:  同意。下一輪我會盡力避免再次犯同樣的錯誤。


第 3 輪：
AI 1 選擇了：布
AI 2 選擇了：石頭
結果：AI 1 贏了！
AI 之間的聊天：
AI 1:  第三輪結束，我

底下程式省略聊天直接以猜拳為主

In [4]:
import google.generativeai as genai
from google.colab import userdata
import random
import time
import re

apikey = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

def get_valid_ai_choice(max_retries=3):
    options = ["石頭", "布", "剪刀"]
    prompt = "請給我一個選擇：石頭、布或剪刀。請務必以清晰的方式回答。"

    previous_choices = []

    for _ in range(max_retries):
        time.sleep(2)
        try:
            response = model.generate_content(prompt)
            choice = response.text.strip()

            match = re.match(r'(石頭|布|剪刀)', choice)
            if match and match.group(0) not in previous_choices:
                previous_choices.append(match.group(0))
                return match.group(0)
            else:
                print(f"AI 選擇了無效的選項：{choice}，請再試一次。")
        except Exception:
            print("AI 發生錯誤，電腦隨機選擇一個選項。")
            return random.choice(options)

    print("超過重試次數，隨機選擇一個選項。")
    return random.choice(options)

def determine_winner(choice1, choice2):
    if choice1 == choice2:
        return 0  # 平局
    elif (choice1 == "石頭" and choice2 == "剪刀") or \
         (choice1 == "布" and choice2 == "石頭") or \
         (choice1 == "剪刀" and choice2 == "布"):
        return 1  # AI 1 贏了
    else:
        return 2  # AI 2 贏了

def ai_vs_ai_rock_paper_scissors(rounds=5):
    ai1_wins = 0
    ai2_wins = 0

    for round_number in range(1, rounds + 1):
        print(f"\n第 {round_number} 輪：")
        time.sleep(1)

        ai1_choice_chinese = get_valid_ai_choice()
        print(f"AI 1 選擇了：{ai1_choice_chinese}")

        ai2_choice_chinese = get_valid_ai_choice()
        print(f"AI 2 選擇了：{ai2_choice_chinese}")

        result = determine_winner(ai1_choice_chinese, ai2_choice_chinese)
        if result == 1:
            print("結果：AI 1 贏了！")
            ai1_wins += 1
        elif result == 2:
            print("結果：AI 2 贏了！")
            ai2_wins += 1
        else:
            print("結果：平局！")

    print("\n遊戲結束！")
    print(f"AI 1 贏了：{ai1_wins} 次, AI 2 贏了：{ai2_wins} 次")

# 開始遊戲
ai_vs_ai_rock_paper_scissors()



第 1 輪：
AI 1 選擇了：石頭
AI 2 選擇了：布
結果：AI 2 贏了！

第 2 輪：
AI 1 選擇了：剪刀
AI 2 選擇了：剪刀
結果：平局！

第 3 輪：
AI 1 選擇了：剪刀
AI 2 選擇了：布
結果：AI 1 贏了！

第 4 輪：
AI 1 選擇了：剪刀
AI 2 選擇了：剪刀
結果：平局！

第 5 輪：
AI 1 選擇了：剪刀
AI 2 選擇了：剪刀
結果：平局！

遊戲結束！
AI 1 贏了：1 次, AI 2 贏了：1 次
